<a href="https://colab.research.google.com/github/atick-faisal/TAVI/blob/main/src/colab/MultiViewUNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colab Info

In [1]:
from psutil import virtual_memory

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)


ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Fri Dec  9 10:34:44 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Download Dataset

In [3]:
!pip install -U --no-cache-dir gdown --pre
!pip install image-similarity-measures > /dev/null

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0


In [4]:
!gdown "12CxxS2w7mprCnVX9Y_Kiti6vEPbTQW2l"
!gdown "1RjmcB2OV5g112ICxQ_VDRkn2zshQeY3C"
!unzip -o "CurvToTAWSSCFDCMAPRealTest.zip" > /dev/null


Downloading...
From: https://drive.google.com/uc?id=12CxxS2w7mprCnVX9Y_Kiti6vEPbTQW2l
To: /content/UNet_2DCNN.py
100% 58.6k/58.6k [00:00<00:00, 63.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1RjmcB2OV5g112ICxQ_VDRkn2zshQeY3C
To: /content/CurvToTAWSSCFDCMAPRealTest.zip
100% 833M/833M [00:17<00:00, 48.6MB/s]


# Imports

In [5]:
import os
import cv2
import random
import datetime
import matplotlib
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm
from numpy.typing import NDArray
from UNet_2DCNN import UNet as UNetSakib
# from image_similarity_measures.quality_metrics import *

matplotlib.use('Agg')
plt.rcParams["font.size"] = 16


# Config

In [6]:
DATASET         = "CurvToTAWSSJetLog"
MODEL_NAME      = "MultiviewUNet_4_32"
DATASET_PATH    = "/content/Images/"
TRAIN_DIR       = "Train/"
TEST_DIR        = "Test/"
INPUT_DIR       = "Input/"
TARGET_DIR      = "Target/"
MODEL_PATH      = "/content/drive/MyDrive/Research/TAVI/Models/"
PRED_PATH       = "/content/drive/MyDrive/Research/TAVI/Predictions/"

IMG_SIZE        = 256
BATCH_SIZE      = 16
BUFFER_SIZE     = 1000
VAL_SPLIT       = 0.2
LEARNING_RATE   = 0.001
N_EPOCHS        = 300
PATIENCE        = 30

EXP_NAME        = f"{MODEL_NAME}_I{IMG_SIZE}_B{BATCH_SIZE}_LR{LEARNING_RATE}_{DATASET}"


# Architecture

In [7]:
class UNet:
    def __init__(
        self,
        img_size: int,
        n_channels: int = 3,
        width: int = 32,
        depth: int = 4,
        kernel_size: int = 3
    ):
        self.img_size = img_size
        self.n_channels = n_channels
        self.width = width
        self.depth = depth
        self.kernel_size = kernel_size

    @staticmethod
    def conv(
        x: tf.Tensor,
        filters: int,
        kernel_size: int
    ) -> tf.Tensor:
        for i in range(2):
            x = tf.keras.layers.Conv2D(
                filters=filters,
                kernel_size=kernel_size,
                strides=1,
                padding="same",
                data_format="channels_last",
                dilation_rate=1,
                groups=1,
                activation=None,
                use_bias=True,
                kernel_initializer="glorot_uniform",
                bias_initializer="zeros"
            )(x)

            x = tf.keras.layers.BatchNormalization()(x)
            x = tf.keras.layers.Activation("relu")(x)

        return x

    @staticmethod
    def deconv(x: tf.Tensor, filters: int) -> tf.Tensor:
        x = tf.keras.layers.Conv2DTranspose(
            filters=filters,
            kernel_size=2,
            strides=2,
            padding="same",
            output_padding=None,
            data_format=None,
            dilation_rate=1,
            activation=None,
            use_bias=True,
            kernel_initializer="glorot_uniform",
            bias_initializer="zeros"
        )(x)

        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation("relu")(x)

        return x

    @staticmethod
    def output(x: tf.Tensor) -> tf.Tensor:
        return tf.keras.layers.Conv2D(3, (1, 1), activation="sigmoid")(x)

    @staticmethod
    def pool(x: tf.Tensor, pool_size: int = 2) -> tf.Tensor:
        return tf.keras.layers.MaxPool2D(pool_size)(x)

    def __call__(self) -> tf.keras.Model:
        inputs = tf.keras.layers.Input(
            shape=(self.img_size, self.img_size, self.n_channels)
        )

        # scaled = tf.keras.layers.Rescaling(1./255.0, offset=0)(inputs)

        # ------------------ Downsampling ---------------------
        downsample_layers = []
        downsample_layers.append(
            self.conv(
                x=inputs,
                filters=self.width,
                kernel_size=self.kernel_size
            )
        )
        for i in range(1, self.depth):
            filters = int((2 ** i) * self.width)
            downsample_layers.append(
                self.pool(
                    self.conv(
                        x=downsample_layers[i - 1],
                        filters=filters,
                        kernel_size=self.kernel_size
                    )
                )
            )

        # ------------------- Features --------------------
        n_features = int((2 ** self.depth) * self.width)
        self.features = self.pool(
            self.conv(
                x=downsample_layers[-1],
                filters=n_features,
                kernel_size=self.kernel_size
            )
        )

        # ------------------- Upsampling --------------------
        upsample_layers = []
        upsample_layers.append(self.features)
        for i in range(1, self.depth + 1):
            filters = int((2 ** (self.depth - i)) * self.width)
            upsample_layers.append(
                self.conv(
                    x=tf.keras.layers.concatenate([
                        downsample_layers[self.depth - i],
                        self.deconv(
                            x=upsample_layers[i - 1],
                            filters=filters
                        )
                    ]),
                    filters=filters,
                    kernel_size=self.kernel_size
                )
            )

        # ---------------------- Output -----------------------
        outputs = self.output(upsample_layers[-1])

        return tf.keras.Model(inputs, outputs)


# Loss Functions

In [8]:
def attention_mse(y_true, y_pred):
    _y_true = y_true[y_true != 1.0]
    _y_pred = y_pred[y_true != 1.0]
    squared_difference = tf.square(_y_true - _y_pred)
    return tf.reduce_mean(squared_difference, axis=-1)


def attention_mae(y_true, y_pred):
    _y_true = y_true[y_true != 1.0]
    _y_pred = y_pred[y_true != 1.0]
    squared_difference = tf.abs(_y_true - _y_pred)
    return tf.reduce_mean(squared_difference, axis=-1)


# Utils

In [9]:
def load_data_from_dir(path: str) -> tf.data.Dataset:
    return tf.keras.utils.image_dataset_from_directory(
        directory=path,
        labels=None,
        color_mode='rgb',
        batch_size=BATCH_SIZE,
        image_size=(IMG_SIZE, IMG_SIZE),
        shuffle=True,
        seed=42,
        # validation_split=VAL_SPLIT,
        # subset=subset,
        interpolation='bilinear',
        follow_links=False,
        crop_to_aspect_ratio=False
    )


# Dataloaders

In [10]:
trainX = load_data_from_dir(os.path.join(DATASET_PATH, TRAIN_DIR, INPUT_DIR))
trainY = load_data_from_dir(os.path.join(DATASET_PATH, TRAIN_DIR, TARGET_DIR))
testX = load_data_from_dir(os.path.join(DATASET_PATH, TEST_DIR, INPUT_DIR))
testY = load_data_from_dir(os.path.join(DATASET_PATH, TEST_DIR, TARGET_DIR))

train_ds = tf.data.Dataset.zip((trainX, trainY))
test_ds = tf.data.Dataset.zip((testX, testY))

print(train_ds.element_spec)
print(test_ds.element_spec)


Found 8280 files belonging to 1 classes.
Found 8280 files belonging to 1 classes.
Found 288 files belonging to 1 classes.
Found 288 files belonging to 1 classes.
(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None))
(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None))


# Normalization

In [11]:
# def normalize(input_image, output_image):
#     input_image = tf.cast(input_image, tf.float32) / 255.0
#     output_image = tf.cast(input_image, tf.float32) / 255.0
#     return input_image, output_image

In [12]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), normalization_layer(y)))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), normalization_layer(y)))

# Augmentation

In [13]:
class Augment(tf.keras.layers.Layer):
    def __init__(self, seed=42):
        super().__init__()
        # both use the same seed, so they'll make the same random changes.
        self.augment_inputs = tf.keras.layers.RandomZoom((-0.1, -0.7), seed=seed)
        self.augment_labels = tf.keras.layers.RandomZoom((-0.1, -0.7), seed=seed)

    def call(self, inputs, labels):
        inputs = self.augment_inputs(inputs)
        labels = self.augment_labels(labels)
        return inputs, labels

# Optimization

In [14]:
AUTOTUNE = tf.data.AUTOTUNE

train_batches = (
    train_ds
    .cache()
    # .shuffle(BUFFER_SIZE)
    # .batch(BATCH_SIZE)
    # .repeat()
    .map(Augment())
    .prefetch(buffer_size=AUTOTUNE)
)

test_batches = (test_ds)


# Training Config

In [15]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=PATIENCE,
        restore_best_weights=True
    )
]

optimizer = tf.keras.optimizers.Adam(
    learning_rate=LEARNING_RATE
)

model = UNet(IMG_SIZE)()
# model = MobileUNet(IMG_SIZE)()
# model = UNetSakib(
#     length=IMG_SIZE, 
#     width=IMG_SIZE, 
#     model_depth=4, 
#     num_channel=3, 
#     model_width=32, 
#     kernel_size=3, 
#     problem_type='Regression',
#     output_nums=3, 
#     ds=0, 
#     ae=0, 
#     ag=0, 
#     lstm=0, 
#     alpha=1, 
#     feature_number=1024, 
#     is_transconv=True
# ).UNet4P()

model.compile(
    loss=attention_mse,
    optimizer=optimizer,
    metrics=[attention_mae]
)


# Training

In [16]:
history = model.fit(
    train_batches,
    validation_data=test_batches,
    epochs=N_EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=callbacks,
    verbose=1
)


Epoch 1/300
518/518 [==============================] - 308s 555ms/step - loss: 0.0138 - attention_mae: 0.0673 - val_loss: 0.0240 - val_attention_mae: 0.0922
Epoch 2/300
518/518 [==============================] - 280s 540ms/step - loss: 0.0114 - attention_mae: 0.0582 - val_loss: 0.0153 - val_attention_mae: 0.0755
Epoch 3/300
518/518 [==============================] - 280s 540ms/step - loss: 0.0108 - attention_mae: 0.0556 - val_loss: 0.0553 - val_attention_mae: 0.1578
Epoch 4/300
518/518 [==============================] - 280s 540ms/step - loss: 0.0104 - attention_mae: 0.0537 - val_loss: 0.0187 - val_attention_mae: 0.0766
Epoch 5/300
518/518 [==============================] - 280s 540ms/step - loss: 0.0102 - attention_mae: 0.0526 - val_loss: 0.0177 - val_attention_mae: 0.0725
Epoch 6/300
518/518 [==============================] - 280s 540ms/step - loss: 0.0098 - attention_mae: 0.0511 - val_loss: 0.0160 - val_attention_mae: 0.0725
Epoch 7/300
518/518 [==============================] - 280

# Save Model

In [17]:
timestamp = datetime.datetime.now().strftime('%b-%d-%I:%M%p')
model_path = os.path.join(MODEL_PATH, EXP_NAME)
if not os.path.exists(model_path):
    os.makedirs(model_path)

model.save(os.path.join(model_path, timestamp))

# Save Predictions

In [18]:
test_ds_unbatched = test_batches.unbatch()

pred_path = os.path.join(PRED_PATH, EXP_NAME, timestamp)
if not os.path.exists(pred_path):
    os.makedirs(pred_path)

metrics = pd.DataFrame()

for idx, (input, target) in enumerate(tqdm(test_ds_unbatched)):

    # if not idx % 10 == 0:
    #     continue

    target = target.numpy()
    prediction = tf.squeeze(
        model.predict(
            tf.expand_dims(input, axis=0)
            )
        ).numpy()
    prediction[target == 1.0] = 1.0

    # _metrics = pd.DataFrame([[
    #         rmse(target, prediction),
    #         psnr(target, prediction),
    #         ssim(target, prediction),
    #         fsim(target, prediction),
    #         issm(target, prediction),
    #         sre(target, prediction),
    #         sam(target, prediction),
    #         uiq(target, prediction),
    #     ]],
    #     columns=["RMSE", "PSNR", "SSIM", "FSIM", "ISSM", "SRE", "SAM", "UIQ"]
    # )

    # metrics = pd.concat([metrics, _metrics], axis=0)

    plt.figure(figsize=(7, 7))
    plt.imshow(target)
    plt.axis("off")
    plt.tight_layout()
    plt.savefig(os.path.join(pred_path, f"{idx}_T.png"))
    plt.close()

    plt.figure(figsize=(7, 7))
    plt.imshow(prediction)
    plt.axis("off")
    plt.tight_layout()
    plt.savefig(os.path.join(pred_path, f"{idx}_P.png"))
    plt.close()


    # plt.figure(figsize=(12, 4))
    # plt.subplot(1, 3, 1)
    # plt.imshow(input)
    # plt.axis("off")
    # plt.title("INPUT")
    # plt.subplot(1, 3, 2)
    # plt.imshow(target)
    # plt.axis("off")
    # plt.title("TARGET")
    # plt.subplot(1, 3, 3)
    # plt.imshow(prediction)
    # plt.axis("off")
    # plt.title("PREDICTION")
    # plt.tight_layout()
    # plt.savefig(os.path.join(pred_path, f"{idx}.png"))
    # plt.close()  


0it [00:00, ?it/s]

1/1 [==============================] - 2s 2s/step


1it [00:02,  2.69s/it]

1/1 [==============================] - 0s 18ms/step


2it [00:03,  1.30s/it]

...

286it [01:34,  3.20it/s]

1/1 [==============================] - 0s 18ms/step


287it [01:34,  3.34it/s]

1/1 [==============================] - 0s 19ms/step


288it [01:34,  3.03it/s]


In [19]:
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 6))
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Attention MAE')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.tight_layout()
plt.savefig(os.path.join(model_path, timestamp + ".png"))
plt.close()

In [20]:
model.evaluate(test_batches)

18/18 [==============================] - 4s 162ms/step - loss: 0.0102 - attention_mae: 0.0466


[0.010194240137934685, 0.04656616598367691]